In [1]:
import pandas as pd
import ccxt
from datetime import datetime, timedelta, timezone
import ta
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import psycopg2
from psycopg2.extras import execute_values

# --- Connexion PostgreSQL ---
conn = psycopg2.connect(
    dbname="trading_scores",
    user="postgres",       # adapte si tu as un autre user
    password="ton_mot_de_passe",  # ⚠️ change
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# --- Instanciation des exchanges ---
exchange_binance = ccxt.binance()
exchange_bitget = ccxt.bitget()
exchange_mexc = ccxt.mexc()

# --- Chargement des marchés ---
def load_symbols(exchange, name, suffix='/USDT'):
    try:
        markets = exchange.load_markets()
        symbols = [s for s in markets if s.endswith(suffix)]
        print(f"[{datetime.now().strftime('%H:%M:%S')}] 📦 {name} : {len(symbols)} paires {suffix}")
        return symbols
    except Exception as e:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] ❌ Erreur chargement {name} : {e}")
        return []

symbols_usdt_all = list(set(
    load_symbols(exchange_binance, 'Binance', '/USDT') +
    load_symbols(exchange_bitget, 'Bitget', '/USDT') +
    load_symbols(exchange_mexc, 'MEXC', '/USDT')
))

symbols_btc_all = list(set(
    load_symbols(exchange_binance, 'Binance', '/BTC') +
    load_symbols(exchange_bitget, 'Bitget', '/BTC') +
    load_symbols(exchange_mexc, 'MEXC', '/BTC')
))

symbol_to_exchange = {s: exchange_binance for s in symbols_usdt_all + symbols_btc_all}
symbol_to_exchange.update({s: exchange_bitget for s in symbols_usdt_all + symbols_btc_all})
symbol_to_exchange.update({s: exchange_mexc for s in symbols_usdt_all + symbols_btc_all})

# --- Timeframes ---
timeframes = {
    '15m': ('15m', 500),
    '1h': ('1h', 500),
    '4h': ('4h', 500),
    '1d': ('1d', 365),
    '1w': ('1w', 150)
}

# --- Récupération OHLCV ---
def fetch_crypto_data(symbol, timeframe, limit, exchange):
    since = exchange.parse8601((datetime.now(timezone.utc) - timedelta(days=365)).strftime('%Y-%m-%dT%H:%M:%S'))
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception:
        return pd.DataFrame()

# --- Ajout indicateurs ---
def add_indicators(df):
    if df.empty or 'close' not in df.columns:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(close=df['close'], window=14).rsi()
    df['SMA200'] = df['close'].rolling(window=200).mean()
    df['Momentum'] = ta.momentum.ROCIndicator(close=df['close'], window=14).roc()
    df['EMA20'] = ta.trend.EMAIndicator(close=df['close'], window=20).ema_indicator()
    df['EMA50'] = ta.trend.EMAIndicator(close=df['close'], window=50).ema_indicator()
    try:
        adx_ind = ta.trend.ADXIndicator(high=df['high'], low=df['low'], close=df['close'], window=14)
        df['ADX'] = adx_ind.adx()
    except:
        df['ADX'] = np.nan
    return df

# --- Dow Theory ---
def dow_trend(df, short_window=50, long_window=200):
    df['High_MA_short'] = df['high'].rolling(window=short_window).mean()
    df['High_MA_long'] = df['high'].rolling(window=long_window).mean()
    df['Low_MA_short'] = df['low'].rolling(window=short_window).mean()
    df['Low_MA_long'] = df['low'].rolling(window=long_window).mean()
    conditions = [
        (df['High_MA_short'] > df['High_MA_long']) & (df['Low_MA_short'] > df['Low_MA_long']),
        (df['High_MA_short'] < df['High_MA_long']) & (df['Low_MA_short'] < df['Low_MA_long'])
    ]
    choices = ['Up', 'Down']
    df['DowTrend'] = np.select(conditions, choices, default='Sideways')
    return df

# --- Scoring ---
def compute_score(row):
    if pd.isna(row['RSI']):
        return None
    score = 0
    score += 10 if row['DowTrend'] == 'Up' else (-10 if row['DowTrend'] == 'Down' else 3)
    score += 5 if row['RSI'] < 30 else (-5 if row['RSI'] > 70 else 0)
    score += 5 if row['Momentum'] > 0 else (-5 if row['Momentum'] < 0 else 0)
    score += 8 if row['ADX'] > 25 else (-8 if row['ADX'] < 20 else 0)
    score += 6 if row['EMA20'] > row['EMA50'] else (-6 if row['EMA20'] < row['EMA50'] else 0)
    score += 5 if row['close'] > row['SMA200'] else (-5 if row['close'] < row['SMA200'] else 0)
    return score

# --- Traitement ---
def fetch_process(sym):
    exchange = symbol_to_exchange.get(sym)
    if not exchange:
        return sym, None
    res = {}
    for tf, (tf_code, limit) in timeframes.items():
        df = fetch_crypto_data(sym, tf_code, limit, exchange)
        if not df.empty:
            df = add_indicators(df)
            df = dow_trend(df)
            df['Score'] = df.apply(compute_score, axis=1)
            res[tf] = df['Score'].iloc[-1] if not df.empty else None
    return sym, res if res else None

def process_symbols(symbols):
    results = []
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=20) as ex:
        futures = {ex.submit(fetch_process, s): s for s in symbols}
        for i, f in enumerate(as_completed(futures), 1):
            sym, row = f.result()
            if row:
                results.append((sym, row))
            if i % 50 == 0:
                elapsed = time.time() - start_time
                print(f"{i} cryptos traitées en {elapsed:.1f}s")
    return results

def build_rows(results, asset_type="crypto"):
    rows = []
    for symbol, scores in results:
        invest = (scores.get('1w', 0) or 0) + (scores.get('1d', 0) or 0)
        swing = (scores.get('4h', 0) or 0) + (scores.get('1h', 0) or 0)
        intra = (scores.get('15m', 0) or 0)
        rows.append((asset_type, symbol, symbol, invest, swing, intra))
    return rows

# --- Sauvegarde PostgreSQL ---
def save_to_postgres(rows):
    query = """
    INSERT INTO assets_scores (asset_type, code, name, score_invest, score_swing, score_intra)
    VALUES %s
    ON CONFLICT (code) DO UPDATE
    SET score_invest = EXCLUDED.score_invest,
        score_swing = EXCLUDED.score_swing,
        score_intra = EXCLUDED.score_intra,
        updated_at = now();
    """
    execute_values(cur, query, rows)
    conn.commit()
    print(f"✅ {len(rows)} enregistrements insérés/maj dans PostgreSQL")

# --- Exécution ---
results_usdt = process_symbols(symbols_usdt_all)
rows_usdt = build_rows(results_usdt, asset_type="crypto")

results_btc = process_symbols(symbols_btc_all)
rows_btc = build_rows(results_btc, asset_type="crypto")

save_to_postgres(rows_usdt + rows_btc)

cur.close()
conn.close()


ModuleNotFoundError: No module named 'psycopg2'

In [2]:
pip install psycopg2-binary

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 26.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install psycopg2-binary